In [1]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../../')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mat73
from scipy.io import loadmat
import os
import time
import collections
from search import Search
from utils import Utils, Struct
from individual import Individual
from initialization import Initialization
import random
random.seed(42)

In [2]:
dfs = []
repeated_columns = []
unique_columns = []
path = '../../results/permutation/300/'
files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
for file_name in files:
    df = pd.read_csv(path + file_name, delimiter=';', header=0, converters={i: str for i in range(100)})
    print(f"df {file_name} - rows: {len(df)} - columns: {len(df.columns)}")
    
    for column in df.columns:
        repeated_columns.append(column)

    dfs.append(df)
df = pd.concat(dfs, ignore_index=True)
columns_to_add = ["ls_improved_solution", "ls_best_sol", "ls_best_sol_gap", "ls_total_time", "ls_sol_values", "ls_individuals_hashes", "local_search_method", 
"pr_best_sol", "pr_best_sol_gap", "pr_total_time", "pr_sol_values", "pr_individuals_hashes", "path_relinking_method", "improvement_msg"]
for column in columns_to_add:
    if column not in df.columns:
        df[column] = ""

df.to_csv('../../results/permutation/compiled/results_300.csv', index=False, sep=";")
df = pd.read_csv('../../results/permutation/compiled/results_300.csv', delimiter=';', header=0, converters={i: str for i in range(100)})
print(f"df compiled - rows: {len(df)} - columns: {len(df.columns)}, columns added: {len(columns_to_add)}")

df permutation_instances_300-x.csv - rows: 30 - columns: 62
df compiled - rows: 30 - columns: 64, columns added: 14


In [3]:
df.columns

Index(['experiment_id', 'execution_id', 'seed', 'instance', 'n', 'm', 's',
       'max_generations', 'max_generations_without_change', 'max_time',
       'max_time_without_change', 'population_size', 'encoding_method',
       'initialization_method', 'selection_method', 'mutation_method',
       'self_mutation', 'crossover_method', 'parent_selection_method',
       'mutation_probability', 'crossover_probability', 'elite_size',
       'offspring_size', 'perform_path_relinking', 'generate_plots',
       'avoid_clones', 'perform_adaptation', 'perform_local_search',
       'local_search_method', 'best_known_result', 'best_solution_found',
       'gap', 'num_generations', 'total_time_ms', 'start_time', 'finish_time',
       'message', 'ga_best_sol', 'ga_best_sol_gap', 'ga_total_time',
       'ls_improved_solution', 'ls_best_sol', 'ls_best_sol_gap',
       'ls_total_time', 'pr_improved_solution', 'pr_best_sol',
       'pr_best_sol_gap', 'pr_total_time', 'best_sol_changes',
       'best_sol_c

In [6]:
def perform_improvement(path_to_csv, path_to_save, time_to_search, n_individuals, local_search_method = "LSBI", path_relinking_method = "backward"):
    df = pd.read_csv(path_to_csv, delimiter=";", converters={i: str for i in range(100)}) # set the path to the results.csv to get chromosomes to perform improvements file here
    
    for row in df.itertuples():
        improvement_msg = ""
        instance = loadmat(os.path.join("../../instances/", row.instance))
        instance_name = row.instance
        known_result = mat73.loadmat(os.path.join("../../known_results", "x_ls_" + instance_name.replace("Instance_", "")))
        best_known_Z = Search.ldet_objval(instance["A"], known_result["x_ls"].reshape(-1,1))
        start = time.time()
        A = instance["A"]
        n = int(row.n)
        s = int(row.s)
        encoding = "binary"
        env=Struct({"A": A, "n": n, "s": s, "encoding": encoding})
    

        hashes = np.unique(row.improvement_candidates_hashes.split(","))
        chromosomes = [Utils.hash_2_chromosome(h, n) for h in hashes][:n_individuals]
        msg = ""
        stop = False
        for c in chromosomes:
            if np.array(list(c)).shape[0] != n:
                msg = f"Error: chromosome shape is not correct at row Index: {row.Index} - c_shape: {np.array(list(c)).shape[0]} - n: {n}"
                stop = True
                break
        if stop:
            print(msg)
            continue
        
        # raw_chromosomes =  row.improvement_candidates.split(",")
        # chromosomes = [np.array(list(c), dtype=int).reshape(-1,1) for c in raw_chromosomes][:n_individuals]
        individuals = [Individual(chromosome=c, environment=env) for c in chromosomes]

        if len(chromosomes) < 1:
            improvement_msg = f"Skiped LS at row with {len(chromosomes)} chromosomes. "
            df.loc[row.Index, "improvement_msg"] += improvement_msg
        else:
            # Local Search
            ls_individuals, ls_solutions = Search.local_search(environment=env, individuals = individuals, max_time = time_to_search, best_sol = best_known_Z, local_search_method = local_search_method)
            df.loc[row.Index, "ls_improved_solution"] = max(ls_solutions) > float(row.best_solution_found)
            df.loc[row.Index, "ls_best_sol"] = max(ls_solutions)
            df.loc[row.Index, "ls_best_sol_gap"] = (max(ls_solutions) - best_known_Z)/abs(best_known_Z)
            df.loc[row.Index, "ls_total_time"] = time_to_search
            df.loc[row.Index, "ls_sol_values"] = ",".join([str(sol) for sol in ls_solutions])
            df.loc[row.Index, "ls_individuals_hashes"] = ",".join([str(ind.individual_hash) for ind in ls_individuals])
            df.loc[row.Index, "local_search_method"] = local_search_method

        if len(chromosomes) < 2:
            improvement_msg = f"Skiped PR at row with {len(chromosomes)} chromosomes. "
            df.loc[row.Index, "improvement_msg"] += improvement_msg
        else:
            # Path Relinking
            pr_hashes, pr_solutions, pr_time = Search.path_relinking(chromosomes, A, path_relinking_method)
            df.loc[row.Index, "pr_improved_solution"] = max(pr_solutions) > float(row.best_solution_found)
            df.loc[row.Index, "pr_best_sol"] = max(pr_solutions)
            df.loc[row.Index, "pr_best_sol_gap"] = (max(pr_solutions) - best_known_Z)/abs(best_known_Z)
            df.loc[row.Index, "pr_total_time"] = pr_time
            df.loc[row.Index, "pr_sol_values"] = ",".join([str(s) for s in pr_solutions])
            df.loc[row.Index, "pr_individuals_hashes"] = ",".join([str(h) for h in pr_hashes])
            df.loc[row.Index, "path_relinking_method"] = path_relinking_method
            
        print(f"Row Index: {row.Index} finished in {time.time()-start}.")

    df.to_csv(path_to_save, index=False, sep=";", mode="a", header=True)

path_to_csv = "../../results/permutation/compiled/results_300.csv" # set the path to the results.csv to get chromosomes to perform improvements file here
path_to_save = "../../results/permutation/compiled/results_improved_300.csv"
ls_max_time = 120
perform_improvement(path_to_csv, path_to_save, ls_max_time, 50)

LSBI time limit reached
Row Index: 0 finished in 128.14077520370483.
Row Index: 1 finished in 23.352757930755615.
Row Index: 2 finished in 96.99609160423279.
Row Index: 3 finished in 100.90896606445312.
Row Index: 4 finished in 46.73725342750549.
LSBI time limit reached
Row Index: 5 finished in 134.58346438407898.
Row Index: 6 finished in 69.85502099990845.
Row Index: 7 finished in 124.62969756126404.
LSBI time limit reached
Row Index: 8 finished in 130.14816522598267.
Row Index: 9 finished in 78.112619638443.
Row Index: 10 finished in 73.12362265586853.
Row Index: 11 finished in 94.81520485877991.
Row Index: 12 finished in 94.71136260032654.
Row Index: 13 finished in 70.81560707092285.
Row Index: 14 finished in 46.50009608268738.
LSBI time limit reached
Row Index: 15 finished in 149.30645608901978.
Row Index: 16 finished in 95.7336311340332.
Row Index: 17 finished in 23.35393190383911.
Row Index: 18 finished in 46.49750733375549.
Row Index: 19 finished in 92.79672741889954.
LSBI time 

In [15]:
instance = loadmat(os.path.join("../../instances/", row.instance))
instance_name = row.instance
known_result = mat73.loadmat(os.path.join("../../known_results", "x_ls_" + instance_name.replace("Instance_", "")))
best_known_Z = Search.ldet_objval(instance["A"], known_result["x_ls"].reshape(-1,1))

In [7]:
row = df.iloc[0]

In [16]:
best_known_Z

4.3357665985157565